### Downloading and installing the necessary libraries

In [ ]:
#!pip install lxml
#!conda install -c anaconda beautifulsoup4

### Importing the necessary libraries

In [2]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

### Use the GET and BEAUTIFULSOUP methods to obtain the raw html content of the url

In [26]:
res = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(res.content,'lxml')

### Use the find_all() method to extract the table contents and store it in a dataframe

In [27]:
table = soup.find_all('table')[0]
df = pd.read_html(str(table))[0]

### What is the shape and the columns of the dataframe?

In [28]:
print(df.shape)
print(df.columns)

(180, 3)
Index(['Postal Code', 'Borough', 'Neighbourhood'], dtype='object')


### We need to remove the rows that do not have an assigned Borough.

Check the unique values of Borough

In [29]:
df['Borough'].unique()

array(['Not assigned', 'North York', 'Downtown Toronto', 'Etobicoke',
       'Scarborough', 'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

If the Borough value is "Not assigned" - remove it from the dataframe

In [30]:
df = df[df['Borough'] != 'Not assigned']
df['Borough'].unique()

array(['North York', 'Downtown Toronto', 'Etobicoke', 'Scarborough',
       'East York', 'York', 'East Toronto', 'West Toronto',
       'Central Toronto', 'Mississauga'], dtype=object)

 Now check the updated shape and number of rows of the dataframe

In [31]:
print(df.shape)

(103, 3)


### Check for any "Not assigned" Neighbourhoods

In [33]:
df['Neighbourhood'].unique().tolist()

['Parkwoods',
 'Victoria Village',
 'Regent Park, Harbourfront',
 'Lawrence Manor, Lawrence Heights',
 "Queen's Park, Ontario Provincial Government",
 'Islington Avenue, Humber Valley Village',
 'Malvern, Rouge',
 'Don Mills',
 'Parkview Hill, Woodbine Gardens',
 'Garden District, Ryerson',
 'Glencairn',
 'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale',
 'Rouge Hill, Port Union, Highland Creek',
 'Woodbine Heights',
 'St. James Town',
 'Humewood-Cedarvale',
 'Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood',
 'Guildwood, Morningside, West Hill',
 'The Beaches',
 'Berczy Park',
 'Caledonia-Fairbanks',
 'Woburn',
 'Leaside',
 'Central Bay Street',
 'Christie',
 'Cedarbrae',
 'Hillcrest Village',
 'Bathurst Manor, Wilson Heights, Downsview North',
 'Thorncliffe Park',
 'Richmond, Adelaide, King',
 'Dufferin, Dovercourt Village',
 'Scarborough Village',
 'Fairview, Henry Farm, Oriole',
 'Northwood Park, York University',
 'East Toronto, Broadview Nort

There happens to be no Neighbour that is "not assigned". So we need to remove any Neighbourhood.

On a side-note: It is possible that a Borough can belong to multiple Neighbourhoods as seen in the "Neighbourhood" column here.

### What is the final Shape of the dataframe?

In [38]:
print(df.shape)

(103, 3)


### Uploading the Latitude and Longitude values as a CSV
### Creating a common Dataframe with the coordinates of the (Borough x Neighbourhood)

In [40]:
df_coor = pd.read_csv("Geospatial_Coordinates.csv")
df_coor.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Let's check the structufe of the "df" and "df_coor" dataframes to understand how to join and obtain the common dataframe.

In [41]:
print(df_coor.head())
print(df.head())

  Postal Code   Latitude  Longitude
0         M1B  43.806686 -79.194353
1         M1C  43.784535 -79.160497
2         M1E  43.763573 -79.188711
3         M1G  43.770992 -79.216917
4         M1H  43.773136 -79.239476
  Postal Code           Borough                                Neighbourhood
2         M3A        North York                                    Parkwoods
3         M4A        North York                             Victoria Village
4         M5A  Downtown Toronto                    Regent Park, Harbourfront
5         M6A        North York             Lawrence Manor, Lawrence Heights
6         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government


Let's use merge() to obtain the common dataframe

In [43]:
df_final = pd.merge(df, df_coor, on = 'Postal Code')

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


Checking whether the "merge" has happened successfully.

In [46]:
print(df_final.shape)
display(df_final.head())

(103, 5)


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


We have obtained the required dataframe. The merge is successful!